In [ ]:
import os
import pandas as pd
from tqdm import tqdm_notebook as tqdm
from multiprocessing import Pool

import requests
from io import open as iopen
from urllib.parse import urlsplit

import warnings
warnings.filterwarnings('ignore')


from PIL import Image
import imghdr


# PATH_TO_DATA="/Volumes/HDD/Datasets/Курсач/vgg_face_dataset/"
# PATH_TO_LABELS="/Volumes/HDD/Datasets/Курсач/vgg_face_dataset/files/_all_in_one_10.csv"
PATH_TO_DATA="/Volumes/HDD/Datasets/Курсач/vgg_face_dataset/files/"
PATH_TO_DOWNLOAD="/Volumes/HDD/Datasets/Курсач/vgg_face_dataset/images_10/"
OUT_FILE="_all_in_one_10.csv"
PAHT_TO_CROPED="/Volumes/HDD/Datasets/Курсач/vgg_face_dataset/images_10_crop/"

### Создание единого файла с сылками

In [ ]:
def accumulate_n_files_in_one(num_img_per_person,out_file_name,path_to_data):
    '''создает один файл в котором num_img_per_person изображений для кажго человека'''
    
    if os.path.isfile(path_to_data+out_file_name): 
        os.remove(path_to_data+out_file_name)
        
    for file in tqdm(os.listdir(path_to_data)):
        name,ext=os.path.splitext(file)
        if ext==".txt":
            tmp=pd.read_csv(path_to_data+file,sep=" ",header=None,nrows=num_img_per_person)
            tmp[0]=name
            tmp.to_csv(path_to_data+out_file_name,mode="a",index=False,header=False)
            
    df=pd.read_csv(path_to_data+out_file_name,header=None)
    df["num_image"]=df.index
    df.to_csv(path_to_data+out_file_name,mode="w",index=False)

In [ ]:
accumulate_n_files_in_one(10,OUT_FILE,PATH_TO_DATA)

### Скачивание изображений из единого файла

In [ ]:
def download_image(row):
    OUT_PATH=PATH_TO_DOWNLOAD #Хакдкод!!!!!!
    file_url,name = row
    try:
        i = requests.get(file_url,timeout=0.4)
    except requests.exceptions.RequestException as e:
        return
    if i.status_code == requests.codes.ok:
        with iopen(OUT_PATH+str(name)+".jpg", 'wb') as file:
            file.write(i.content)

In [ ]:
df=pd.read_csv(PATH_TO_DATA+OUT_FILE)

In [ ]:
# for row in tqdm(zip(df['1'], df['num_image'])):
#     download_image(row)

In [ ]:
with Pool() as p:
    p.map(download_image, zip(df['1'], df['num_image']))

### Обрезка фото (нужны только лица)

In [ ]:
def crop_face(path_to_labels,path_to_images,path_to_new_folder):
    df=pd.read_csv(path_to_labels)
    
    for file in tqdm(os.listdir(path_to_images)):
        name,ext=os.path.splitext(file)
        if ext==".jpg":
            
            checker=imghdr.what(path_to_images+file)
            if checker =="jpeg":
                img = Image.open(path_to_images+file)
                
                area=df[df["num_image"]==int(name)][["2","3","4","5"]].values[0]
                area = (area[0],area[1],area[2],area[3])
                
                cropped_img = img.crop(area)
                cropped_img.save(path_to_new_folder+file)

In [ ]:
crop_face(PATH_TO_DATA+OUT_FILE,PATH_TO_DOWNLOAD,PAHT_TO_CROPED)